In [2]:
import csv
import json
import mysql.connector

# please modify hw4_config.json to your own config file
with open('hw5_config.json') as f:
    config = json.load(f)

host = config['host']
user = config['user']
passwd = config['passwd']
dbms_student_path = config['dbms_student_path']
new_student_path = config['new_student_path']

mydb = mysql.connector.connect(host=host, user=user, passwd=passwd)
cursor = mydb.cursor()

In [3]:
DB_Name = "DB_class"
cursor.execute("DROP DATABASE IF EXISTS " + DB_Name)
cursor.execute("CREATE DATABASE " + DB_Name)

cursor.execute("USE " + DB_Name)
mydb.commit()

## 1-1: Input data into student table

In [4]:
remove_table = "DROP TABLE IF EXISTS Student;"
cursor.execute(remove_table)
mydb.commit()

create_table = ''' 
    CREATE TABLE IF NOT EXISTS Student (
        身份 VARCHAR(16) NOT NULL,
        系所 VARCHAR(32) ,
        年級 INT DEFAULT 1,
        學號 VARCHAR(16) NOT NULL,
        姓名 VARCHAR(64) NOT NULL,
        信箱 VARCHAR(64) NOT NULL,
        班別 VARCHAR(32) NOT NULL,
        PRIMARY KEY(學號)
    );
'''

cursor.execute(create_table)
mydb.commit()

with open(dbms_student_path, encoding="Big5") as csvfile:
    rows = csv.reader(csvfile, delimiter=',')
    next(rows)
    for row in rows:
        row[2] = int(row[2])
        cursor.execute("INSERT INTO Student VALUES (%s, %s, %s, %s, %s, %s, %s)", row)

    mydb.commit()

cursor.execute("SELECT * FROM Student")

result = cursor.fetchall()
for row in result:
    print(row)

('學生', '電機系      ', 4, 'B07901113', '廖甜雅 (LIAO, TIEN-YA)', 'b07901113@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '生物機電系  ', 4, 'B08611041', '郭子敬 (TZU-CHING KUO)', 'b08611041@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '經濟系      ', 4, 'B09303019', '黃于軒 (HUANG,YU-hsUAN)', 'b09303019@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '經濟系      ', 4, 'B09303021', '李胤愷 (Lee, yin-kai)', 'b09303021@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '經濟系      ', 4, 'B09303027', '林睿霖 (LIN,RUEI-LIN)', 'b09303027@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '經濟系      ', 4, 'B09303090', '呂彥欣 (LU,YAN-XIN)', 'b09303090@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '機械系      ', 4, 'B09502132', '周哲瑋 (CHOU,CHE-WEI)', 'b09502132@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '材料系      ', 4, 'B09507021', '王禹翔 (WANG YU-HSIANG)', 'b09507021@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '醫工系      ', 4, 'B09508013', '陳品文 (CHEN, PIN-WEN)', 'b09508013@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)'

## 1-2: Read the information about yourself

In [5]:
cursor.execute("SELECT * FROM Student WHERE 學號 = 'R10625016'")

result = cursor.fetchall()
for row in result:
    print(row)

('學生', '森林環資所  ', 3, 'R10625016', '許致銓 (CHIH-CHUAN HSU)', 'r10625016@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-3: Update your own value of the "身份" column 

In [6]:
# update 
cursor.execute("UPDATE Student SET 身份 = '特優生' WHERE 學號 = 'R10625016'")
mydb.commit()

cursor.execute("SELECT * FROM Student WHERE 學號 = 'R10625016'")
result = cursor.fetchall()
for row in result:
    print(row)

('特優生', '森林環資所  ', 3, 'R10625016', '許致銓 (CHIH-CHUAN HSU)', 'r10625016@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-4: Insert three new students 

In [7]:
with open(new_student_path, encoding='utf-8') as csvfile:
    rows = csv.reader(csvfile, delimiter=',')
    next(rows)
    for row in rows:
        row[2] = int(row[2])
        cursor.execute("INSERT INTO Student VALUES (%s, %s, %s, %s, %s, %s, %s)", row)

    mydb.commit()

cursor.execute("SELECT * FROM Student WHERE 姓名 = '小紅' OR 姓名 = '小黃' OR 姓名 = '小綠'")
result = cursor.fetchall()
for row in result:
    print(row)

('學生', '物理系', 3, 'B09987653', '小黃', 'b09987653@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('旁聽生', '電機所', 2, 'R10123456', '小紅', 'r10123456@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('觀察者', '電信所', 1, 'R11123001', '小綠', 'r11123001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-5: Prepared select statement 

In [8]:
prepared_cursor = mydb.cursor(prepared=True)

query = "SELECT * FROM Student WHERE 姓名 = %s"
student_names = ['小紅', '小黃', '小綠']
for name in student_names:
    prepared_cursor.execute(query, (name,))
    result = prepared_cursor.fetchall()
    for row in result:
        print(row)

('旁聽生', '電機所', 2, 'R10123456', '小紅', 'r10123456@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '物理系', 3, 'B09987653', '小黃', 'b09987653@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('觀察者', '電信所', 1, 'R11123001', '小綠', 'r11123001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-6: Group information

In [9]:
add_column = '''
    ALTER TABLE Student 
    ADD COLUMN `group` INT DEFAULT 0,
    ADD COLUMN `captain` VARCHAR(1) DEFAULT 'N';
'''

# check if the column exists: if yes, delete the column
try:
    cursor.execute("ALTER TABLE Student DROP COLUMN `group`")
    cursor.execute("ALTER TABLE Student DROP COLUMN `captain`")
except:
    pass

cursor.execute(add_column)

update_my_group = '''
    UPDATE Student 
    SET `group` = 9
    WHERE 學號 = 'R10625016' OR 學號 = 'F10921065' OR 學號 = 'R12922064' OR 學號 = 'B09901162';
'''

cursor.execute(update_my_group)

update_my_captain = '''
    UPDATE Student 
    SET `captain` = 'Y'
    WHERE 學號 = 'R10625016';
'''

cursor.execute(update_my_captain)

select_my_group = '''
    SELECT * FROM Student WHERE `group` = 9;
'''

cursor.execute(select_my_group)
result = cursor.fetchall()
for row in result:
    print(row)


mydb.commit()

('學生', '電機系      ', 4, 'B09901162', '陳冠霖 (CHEN,GUAN-LIN)', 'b09901162@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 9, 'N')
('學生', '', 2, 'F10921065', '吳建翰 (WU, JIAN-HAN)', 'f10921065@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 9, 'N')
('特優生', '森林環資所  ', 3, 'R10625016', '許致銓 (CHIH-CHUAN HSU)', 'r10625016@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 9, 'Y')
('學生', '資工所      ', 1, 'R12922064', '林郁敏 (LIN, YU-MIN)', 'r12922064@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 9, 'N')


In [10]:
# DB_Name = "DB_class"
# drop_db = "DROP DATABASE IF EXISTS " + DB_Name
# cursor.execute(drop_db)
# mydb.commit()
cursor.close()
mydb.close()